The **Monte Carlo** methods of evaluating conditional mean for option pricing is nearly always the method of choice for high-dimensional problems. It is also easy to apply to problems with non-Markovian dynamics as well as strong path-dependency in the payout, which PDE or finite-difference method cannot handle. However, it is not straightforward to apply Monte Carlo on backward induction problem, such as American or Bermudan option pricing.

The fundamental theoretical foundation of Monte Carlo is the **Strong Law of Large Numbers (LLN)** and **Central Limit Theorem (CLT)**.

## Generating Random Variables

### Inverse Transform

Let $Z\sim F^{-1}(U)$, then $Z$ has CDF $F$. One can generalize by using general inverse of $F$ rather than the true inverse.

### Acceptance-Rejection Method

For the target pdf $f(z)$, find a 'blanket' $e(z)$ and constant $C$ such that $Ce(z)\geq f(z)$, and that $e(z)$ is easy to sample from.
- Draw a sample $Z$ from $e(z)$
- Draw an independent uniform variable $U$
- Accept the sample $Z$ if $U\leq f(Z)/(Ce(Z))$; otherwise reject it.

$C$ should be chosen as tight as possible so that it does not incur wasteful rejections.

### Composition

This is just a fancy name to use known mapping between random variables to generate from one to another, e.g. from Gaussian to log normal. Another example is to use linear transformation and cholesky decomposition to go from standardized multi-dimensional Gaussian to general multi-dimensional Gaussian.